In [1]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os
import numpy as np
from tensorflow.keras.preprocessing import image

In [2]:
# === Dataset path ===
train_dir = r'C:\Hope\AI Course Tamil\8.Deep Learning\dataset'

# === Data Augmentation ===

# 🧠 Overfitting Prevention:
datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2)

# 🎯 Generalization Improve Aagum:
train_gen = datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),  # MobileNetV2 input size
    batch_size=32,
    class_mode='binary',
    subset='training')

# 📈 Virtual Dataset Expansion:
val_gen = datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary',
    subset='validation')

# === Print class labels ===
print("Class labels:", train_gen.class_indices)

Found 2145 images belonging to 2 classes.
Found 535 images belonging to 2 classes.
Class labels: {'hyena': 0, 'not_hyene': 1}


In [3]:
# === Load Pretrained Model (MobileNetV2) ===
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
base_model.trainable = False  # Freeze base model layers

In [4]:
# === Add custom classifier on top ===
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(64, activation='relu')(x)
predictions = Dense(1, activation='sigmoid')(x)  # Binary classification

model = Model(inputs=base_model.input, outputs=predictions)

# === Compile the model ===
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [5]:
# === Train the model ===
model.fit(train_gen, validation_data=val_gen, epochs=5)



C:\Anaconda3\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/5
68/68 ━━━━━━━━━━━━━━━━━━━━ 105s 1s/step - accuracy: 0.8852 - loss: 0.2496 - val_accuracy: 0.9813 - val_loss: 0.0680
Epoch 2/5
68/68 ━━━━━━━━━━━━━━━━━━━━ 92s 1s/step - accuracy: 0.9976 - loss: 0.0112 - val_accuracy: 0.9738 - val_loss: 0.0783
Epoch 3/5
68/68 ━━━━━━━━━━━━━━━━━━━━ 97s 1s/step - accuracy: 1.0000 - loss: 0.0076 - val_accuracy: 0.9757 - val_loss: 0.0768
Epoch 4/5
68/68 ━━━━━━━━━━━━━━━━━━━━ 95s 1s/step - accuracy: 1.0000 - loss: 0.0023 - val_accuracy: 0.9776 - val_loss: 0.0730
Epoch 5/5
68/68 ━━━━━━━━━━━━━━━━━━━━ 101s 1s/step - accuracy: 1.0000 - loss: 0.0013 - val_accuracy: 0.9794 - val_loss: 0.0712


In [6]:
# === Save the model ===
model.save(r'C:\Hope\AI Course Tamil\8.Deep Learning\hyena_mobilenetv2.h5')
print("✅ Model saved!")

✅ Model saved!


In [7]:
# === Load and test on a single image ===
model = tf.keras.models.load_model(r'C:\Hope\AI Course Tamil\8.Deep Learning\hyena_mobilenetv2.h5')

img_path = r'C:\Hope\AI Course Tamil\8.Deep Learning\test_image.jpg'
if not os.path.exists(img_path):
    print("Image not found:", img_path)
else:
    img = image.load_img(img_path, target_size=(224, 224))
    img_array = image.img_to_array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)

    prediction = model.predict(img_array)
    score = prediction[0][0]
    percentage = score * 100

    print(f"Raw Prediction Score: {score:.4f}")
    if score > 0.5:
        print(f"🦁 Prediction: Not a Hyena ({percentage:.2f}%)")
    else:
        print(f"🦝 Prediction: Hyena Detected ({100 - percentage:.2f}%)")

1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
Raw Prediction Score: 0.0000
🦝 Prediction: Hyena Detected (100.00%)
